In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns

In [2]:
#Read files:
df = pd.read_csv("clean_1_train.csv")
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8519 entries, 0 to 8518
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8519 non-null   object 
 1   Item_Weight                8519 non-null   float64
 2   Item_Fat_Content           8519 non-null   object 
 3   Item_Visibility            8519 non-null   float64
 4   Item_Type                  8519 non-null   object 
 5   Item_MRP                   8519 non-null   float64
 6   Outlet_Identifier          8519 non-null   object 
 7   Outlet_Establishment_Year  8519 non-null   int64  
 8   Outlet_Size                8519 non-null   object 
 9   Outlet_Location_Type       8519 non-null   object 
 10  Outlet_Type                8519 non-null   object 
 11  Item_Outlet_Sales          8519 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 798.8+ KB


In [4]:
df.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [5]:
df.nunique()

Item_Identifier              1555
Item_Weight                   474
Item_Fat_Content                5
Item_Visibility              7876
Item_Type                      16
Item_MRP                     5936
Outlet_Identifier              10
Outlet_Establishment_Year       9
Outlet_Size                     3
Outlet_Location_Type            3
Outlet_Type                     4
Item_Outlet_Sales            3493
dtype: int64

In [6]:
# Displaying the mean sales by type of store.
df.pivot_table(values='Item_Outlet_Sales',index='Outlet_Type')

,Item_Outlet_Sales
Outlet_Type,
Grocery Store,340.031198
Supermarket Type1,2316.181148
Supermarket Type2,1995.498739
Supermarket Type3,3695.781505


#### Modify Item_Visibility
We noticed that the minimum value here is 0, which makes no practical sense.

In [7]:
def modify_item_visibility(df):
    #Determine average visibility of a product
    visibility_avg = df.pivot_table(values='Item_Visibility', index='Item_Identifier')

    #Impute 0 values with mean visibility of that product:
    miss_bool = (df['Item_Visibility'] == 0)

    print('Number of 0 values initially: %d'%sum(miss_bool))
    df.loc[miss_bool,'Item_Visibility'] = df.loc[miss_bool,'Item_Identifier'].apply(lambda x: visibility_avg.loc[x])
    print('Number of 0 values after modification: %d'%sum(df['Item_Visibility'] == 0))

In [8]:
modify_item_visibility(df)

Number of 0 values initially: 526
Number of 0 values after modification: 0


In [9]:
df["Item_Visibility"].describe()

count    8519.000000
mean        0.069652
std         0.049798
min         0.003575
25%         0.031114
50%         0.056919
75%         0.097132
max         0.328391
Name: Item_Visibility, dtype: float64

In [10]:
df['Item_Identifier']

0       FDA15
1       DRC01
2       FDN15
3       FDX07
4       NCD19
        ...  
8514    FDF22
8515    FDS36
8516    NCJ29
8517    FDN46
8518    DRG01
Name: Item_Identifier, Length: 8519, dtype: object

In [11]:
print('Frequency of Categories for varible Item Type')
df['Item_Type'].value_counts()

Frequency of Categories for varible Item Type


Fruits and Vegetables    1232
Snack Foods              1199
Household                 910
Frozen Foods              855
Dairy                     681
Canned                    649
Baking Goods              647
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64

In [12]:
df.groupby(["Item_Identifier","Item_Type"]).size()

Item_Identifier  Item_Type         
DRA12            Soft Drinks           6
DRA24            Soft Drinks           7
DRA59            Soft Drinks           8
DRB01            Soft Drinks           3
DRB13            Soft Drinks           5
                                      ..
NCZ30            Household             7
NCZ41            Health and Hygiene    5
NCZ42            Household             5
NCZ53            Health and Hygiene    5
NCZ54            Household             7
Length: 1555, dtype: int64

We can clearly observe that the First 2 characters of the Item ID is same for the One kind of Item Type.
Example: DR is the code for Soft Drinks, NC is the code of Non- Consumable Products and FD is for Food products

#### Create a broad category of Type of Item

In [13]:
def broad_item_type(df):
    #Get the first two characters of ID:
    df['Item_Type_Combined'] = df['Item_Identifier'].apply(lambda x: x[0:2])
    #Rename them to more intuitive categories:
    df['Item_Type_Combined'] = df['Item_Type_Combined'].map({'FD':'Food',
                                                                 'NC':'Non-Consumable',
                                                                 'DR':'Drinks'})
    print(df['Item_Type_Combined'].value_counts())

In [14]:
broad_item_type(df)

Food              6121
Non-Consumable    1599
Drinks             799
Name: Item_Type_Combined, dtype: int64


### Determine the years of operation of a store 

In [15]:
df["Outlet_Establishment_Year"].value_counts()

1985    1459
1987     932
1999     930
1997     930
2004     930
2002     929
2009     928
2007     926
1998     555
Name: Outlet_Establishment_Year, dtype: int64

In [16]:
df.groupby(["Outlet_Establishment_Year","Outlet_Identifier","Outlet_Type","Outlet_Location_Type"])["Item_Outlet_Sales"].mean()

Outlet_Establishment_Year  Outlet_Identifier  Outlet_Type        Outlet_Location_Type
1985                       OUT019             Grocery Store      Tier 1                   340.746838
                           OUT027             Supermarket Type3  Tier 3                  3695.781505
1987                       OUT013             Supermarket Type1  Tier 3                  2298.995256
1997                       OUT046             Supermarket Type1  Tier 1                  2277.844267
1998                       OUT010             Grocery Store      Tier 3                   339.351662
1999                       OUT049             Supermarket Type1  Tier 1                  2348.354635
2002                       OUT045             Supermarket Type1  Tier 2                  2192.384798
2004                       OUT035             Supermarket Type1  Tier 2                  2438.841866
2007                       OUT017             Supermarket Type1  Tier 2                  2340.675263
2009 

In [17]:
def cal_outlet_year(df):
    #Years:
    df['Outlet_Years'] = 2013 - df['Outlet_Establishment_Year']
    print(df['Outlet_Years'].describe())

In [18]:
cal_outlet_year(df)

count    8519.000000
mean       15.162108
std         8.369105
min         4.000000
25%         9.000000
50%        14.000000
75%        26.000000
max        28.000000
Name: Outlet_Years, dtype: float64


### Modify categories of Item_Fat_Content
We found typos and difference in representation in categories of Item_Fat_Content variable.

In [19]:
def modify_item_fat_content(data):
    #Change categories of low fat:
    print('Original Categories:')
    print(data['Item_Fat_Content'].value_counts())

    print('\nModified Categories:')
    data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                                 'reg':'Regular',
                                                                 'low fat':'Low Fat'})
    print(data['Item_Fat_Content'].value_counts())

In [20]:
modify_item_fat_content(df)

Original Categories:
Low Fat    5088
Regular    2886
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

Modified Categories:
Low Fat    5516
Regular    3003
Name: Item_Fat_Content, dtype: int64


#### We have corrected the typos and the difference in representation but some non-consumables as well and a fat-content should not be specified for them. So we can also create a separate category for such kind of observations.

In [21]:
def non_consumable_category(data):
    #Mark non-consumables as separate category in low_fat:
    data.loc[data['Item_Type_Combined']=="Non-Consumable",'Item_Fat_Content'] = "Non-Edible"
    print(data['Item_Fat_Content'].value_counts())

In [22]:
non_consumable_category(df)

Low Fat       3917
Regular       3003
Non-Edible    1599
Name: Item_Fat_Content, dtype: int64


In [23]:
def Item_Visibility_MeanRatio(data):
    #Get all Item_Visibility mean values for respective Item_Identifier
    visibility_item_avg = data.pivot_table(values='Item_Visibility',index='Item_Identifier')

    func = lambda x: x['Item_Visibility']/visibility_item_avg['Item_Visibility'][visibility_item_avg.index == x['Item_Identifier']][0]
    data['Item_Visibility'] = data.apply(func,axis=1).astype(float)
    data['Item_Visibility'].describe()
    

In [24]:
Item_Visibility_MeanRatio(df)

In [25]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Type_Combined,Outlet_Years
0,FDA15,9.30,Low Fat,0.922960,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,Food,14
1,DRC01,5.92,Regular,1.003057,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,Drinks,4
2,FDN15,17.50,Low Fat,0.831990,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,Food,14
3,FDX07,19.20,Regular,0.750000,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800,Food,15
4,NCD19,8.93,Non-Edible,0.666667,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,Non-Consumable,26


In [26]:
df.shape

(8519, 14)

# Numerical and One-Hot Coding of Categorical variables

In [27]:
df.nunique()

Item_Identifier              1555
Item_Weight                   474
Item_Fat_Content                3
Item_Visibility              8032
Item_Type                      16
Item_MRP                     5936
Outlet_Identifier              10
Outlet_Establishment_Year       9
Outlet_Size                     3
Outlet_Location_Type            3
Outlet_Type                     4
Item_Outlet_Sales            3493
Item_Type_Combined              3
Outlet_Years                    9
dtype: int64

# Label Encoding

In [28]:
#Import library:
from sklearn.preprocessing import LabelEncoder

def label_encoding(df):
    le = LabelEncoder()
    #New variable for outlet
    df['Outlet'] = le.fit_transform(df['Outlet_Identifier'])
    df['Outlet']
    var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
    le = LabelEncoder()
    for i in var_mod:
        df[i] = le.fit_transform(df[i])

In [29]:
label_encoding(df)

In [30]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Type_Combined,Outlet_Years,Outlet
0,FDA15,9.30,0,0.922960,Dairy,249.8092,OUT049,1999,1,0,1,3735.1380,1,14,9
1,DRC01,5.92,2,1.003057,Soft Drinks,48.2692,OUT018,2009,1,2,2,443.4228,0,4,3
2,FDN15,17.50,0,0.831990,Meat,141.6180,OUT049,1999,1,0,1,2097.2700,1,14,9
3,FDX07,19.20,2,0.750000,Fruits and Vegetables,182.0950,OUT010,1998,2,2,0,732.3800,1,15,0
4,NCD19,8.93,1,0.666667,Household,53.8614,OUT013,1987,0,2,1,994.7052,2,26,1


In [31]:
df.shape

(8519, 15)

In [32]:
def One_hot_encoding(df):
    #One Hot Coding:
    df = pd.get_dummies(df, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type','Item_Type_Combined','Outlet'],drop_first = True)
    
    return df

In [33]:
df = One_hot_encoding(df)

In [34]:
df.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Item_Outlet_Sales,Outlet_Years,Item_Fat_Content_1,...,Item_Type_Combined_2,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,FDA15,9.30,0.922960,Dairy,249.8092,OUT049,1999,3735.1380,14,0,...,0,0,0,0,0,0,0,0,0,1
1,DRC01,5.92,1.003057,Soft Drinks,48.2692,OUT018,2009,443.4228,4,0,...,0,0,0,1,0,0,0,0,0,0
2,FDN15,17.50,0.831990,Meat,141.6180,OUT049,1999,2097.2700,14,0,...,0,0,0,0,0,0,0,0,0,1
3,FDX07,19.20,0.750000,Fruits and Vegetables,182.0950,OUT010,1998,732.3800,15,0,...,0,0,0,0,0,0,0,0,0,0
4,NCD19,8.93,0.666667,Household,53.8614,OUT013,1987,994.7052,26,1,...,1,1,0,0,0,0,0,0,0,0


In [35]:
df.shape

(8519, 29)

In [36]:
df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Item_Outlet_Sales            float64
Outlet_Years                   int64
Item_Fat_Content_1             uint8
Item_Fat_Content_2             uint8
Outlet_Location_Type_1         uint8
Outlet_Location_Type_2         uint8
Outlet_Size_1                  uint8
Outlet_Size_2                  uint8
Outlet_Type_1                  uint8
Outlet_Type_2                  uint8
Outlet_Type_3                  uint8
Item_Type_Combined_1           uint8
Item_Type_Combined_2           uint8
Outlet_1                       uint8
Outlet_2                       uint8
Outlet_3                       uint8
Outlet_4                       uint8
Outlet_5                       uint8
Outlet_6                       uint8
Outlet_7                       uint8
O

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8519 entries, 0 to 8518
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8519 non-null   object 
 1   Item_Weight                8519 non-null   float64
 2   Item_Visibility            8519 non-null   float64
 3   Item_Type                  8519 non-null   object 
 4   Item_MRP                   8519 non-null   float64
 5   Outlet_Identifier          8519 non-null   object 
 6   Outlet_Establishment_Year  8519 non-null   int64  
 7   Item_Outlet_Sales          8519 non-null   float64
 8   Outlet_Years               8519 non-null   int64  
 9   Item_Fat_Content_1         8519 non-null   uint8  
 10  Item_Fat_Content_2         8519 non-null   uint8  
 11  Outlet_Location_Type_1     8519 non-null   uint8  
 12  Outlet_Location_Type_2     8519 non-null   uint8  
 13  Outlet_Size_1              8519 non-null   uint8

In [38]:
df.to_csv("clean_2_train.csv",index = False)